In [171]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-sentiment-analysis-hatred-speech/train.csv
/kaggle/input/twitter-sentiment-analysis-hatred-speech/test.csv
/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [172]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Different steps for performing text preprocessing in NLP pipeline

## 1) Lowercasing


In [173]:

df['review'] = df['review'].str.lower()
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


## 2) Removing HTML tags using regular expressions i.e regex

In [174]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

df['review'] = df['review'].apply(remove_html_tags)  # apply() is a pandas function used with Series or DataFrame to apply a function to each value.


## 3) Removing URLs using regex


In [175]:

def remove_url(text) :
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)


textl = 'Check out my notebook https://www.kaggle .com/campusx/notebook8223fc1abb'
text2 = 'Check out my notebook http://www.kaggle.com/campusx/notebook8223fc1abb'
text3 ='Google search here www.google.com'
text4 = 'For notebook click https://www.kagg1e.com/campusx/notebook8223fc1abb to search check www.google.com'

print(remove_url(text4))
print("\n")


df['review'] = df['review'].apply(remove_url)


For notebook click  to search check 




## 4) Removing Punctuations


In [176]:

text = ' string ! with : punctuation ,'

import string,time
exclude = string.punctuation
print(exclude,"\n") # all the punctuations which we want to remove

def remove_punc(text):
    for char in exclude:
        text = text.replace(char,'')
    return text

remove_punc(text)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 



' string  with  punctuation '

In [177]:
# time required by python to remove punctuation by using the function we defined

start = time.time()
print(remove_punc(text))
time1 = time.time() - start
print(time1 ,"time to remove the punctuation from given text \n")

print(time1 * 50000 , "time to remove the punctuation from 50000 rows") ## to much time is wasted 

 string  with  punctuation 
0.00024318695068359375 time to remove the punctuation from given text 

12.159347534179688 time to remove the punctuation from 50000 rows


In [178]:
# Proper method to remove punctuation

def remove_punc1(text):
    return text.translate(str.maketrans('','',exclude))


# comparison between two methods
start = time.time()
print(remove_punc1(text))
time2 = time.time() - start
print(time2 ,"time to remove the punctuation from given text \n")

 string  with  punctuation 
0.00026345252990722656 time to remove the punctuation from given text 



In [179]:
# Removing punctuation in twitter hatred speech dataset

df1 = pd.read_csv('/kaggle/input/twitter-sentiment-analysis-hatred-speech/train.csv')
print(df1.head())
print(df1,'\n\n')

df1['tweet'] = df1['tweet'].apply(remove_punc1)
print(df1)


   id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation
          id  label                                              tweet
0          1      0   @user when a father is dysfunctional and is s...
1          2      0  @user @user thanks for #lyft credit i can't us...
2          3      0                                bihday your majesty
3          4      0  #model   i love u take with u all the time in ...
4          5      0             factsguide: society now    #motivation
...      ...    ...                                                ...
31957  31958      0  ate @user isz that youuu?ðððððð...
31958  31959      0    to see nina turner on the

## 5) Chat word treatment


In [180]:
# first finding the meaning of the chatwords

text = """
A3=Anytime, Anywhere, Anyplace
ADIH=Another Day In Hell
AFK=Away From Keyboard
AFAIK=As Far As I Know
ASAP=As Soon As Possible
ASL=Age, Sex, Location
ATK=At The Keyboard
ATM=At The Moment
BAE=Before Anyone Else
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRUH=Bro
BRT=Be Right There
BSAAW=Big Smile And A Wink
BTW=By The Way
BWL=Bursting With Laughter
CSL=Can’t Stop Laughing
CU=See You
CUL8R=See You Later
CYA=See You
DM=Direct Message
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FIMH=Forever In My Heart
FOMO=Fear Of Missing Out
FR=For Real
FWIW=For What It's Worth
FYP=For You Page
FYI=For Your Information
G9=Genius
GAL=Get A Life
GG=Good Game
GMTA=Great Minds Think Alike
GN=Good Night
GOAT=Greatest Of All Time
GR8=Great!
HBD=Happy Birthday
IC=I See
ICQ=I Seek You
IDC=I Don’t Care
IDK=I Don't Know
IFYP=I Feel Your Pain
ILU=I Love You
ILY=I Love You
IMHO=In My Honest/Humble Opinion
IMU=I Miss You
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
IYKYK=If You Know, You Know
JK=Just Kidding
KISS=Keep It Simple, Stupid
L=Loss
L8R=Later
LDR=Long Distance Relationship
LMK=Let Me Know
LMAO=Laughing My A** Off
LOL=Laughing Out Loud
LTNS=Long Time No See
M8=Mate
MFW=My Face When
MID=Mediocre
MRW=My Reaction When
MTE=My Thoughts Exactly
NVM=Never Mind
NRN=No Reply Necessary
NPC=Non-Player Character
OIC=Oh I See
OP=Overpowered
PITA=Pain In The A**
POV=Point Of View
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A** Off
RN=Right Now
SK8=Skate
STATS=Your Sex And Age
SUS=Suspicious
TBH=To Be Honest
TFW=That Feeling When
THX=Thank You
TIME=Tears In My Eyes
TLDR=Too Long, Didn’t Read
TNTL=Trying Not To Laugh
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
W=Win
W8=Wait...
WB=Welcome Back
WTF=What The F**k
WTG=Way To Go!
WUF=Where Are You From?
WYD=What You Doing?
WYWH=Wish You Were Here
ZZZ=Sleeping, Bored, Tired

"""

# Converting these chat words into dictionary format

chat_words = {
    key.strip(): value.strip()
    for key, value in (line.split("=", 1) for line in text.strip().split("\n"))
}

chat_words

{'A3': 'Anytime, Anywhere, Anyplace',
 'ADIH': 'Another Day In Hell',
 'AFK': 'Away From Keyboard',
 'AFAIK': 'As Far As I Know',
 'ASAP': 'As Soon As Possible',
 'ASL': 'Age, Sex, Location',
 'ATK': 'At The Keyboard',
 'ATM': 'At The Moment',
 'BAE': 'Before Anyone Else',
 'BAK': 'Back At Keyboard',
 'BBL': 'Be Back Later',
 'BBS': 'Be Back Soon',
 'BFN': 'Bye For Now',
 'B4N': 'Bye For Now',
 'BRB': 'Be Right Back',
 'BRUH': 'Bro',
 'BRT': 'Be Right There',
 'BSAAW': 'Big Smile And A Wink',
 'BTW': 'By The Way',
 'BWL': 'Bursting With Laughter',
 'CSL': 'Can’t Stop Laughing',
 'CU': 'See You',
 'CUL8R': 'See You Later',
 'CYA': 'See You',
 'DM': 'Direct Message',
 'FAQ': 'Frequently Asked Questions',
 'FC': 'Fingers Crossed',
 'FIMH': 'Forever In My Heart',
 'FOMO': 'Fear Of Missing Out',
 'FR': 'For Real',
 'FWIW': "For What It's Worth",
 'FYP': 'For You Page',
 'FYI': 'For Your Information',
 'G9': 'Genius',
 'GAL': 'Get A Life',
 'GG': 'Good Game',
 'GMTA': 'Great Minds Think Alik

In [181]:
def chat_conversion(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [182]:
chat_conversion( 'GN baby' )

'Good Night baby'

In [183]:
chat_conversion( 'fr i was shocked' )

'For Real i was shocked'

## 6) Spelling Correction


In [184]:

incorrect_text = 'Welcomme tou Nepal. It is veary beutiful coantry'

from textblob import TextBlob

textBlb = TextBlob(incorrect_text)
textBlb.correct().string

'Welcome to Real. It is very beautiful country'

## 7) Removing stopwords


In [185]:

from nltk.corpus import stopwords
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [186]:

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    new_text = []
    for word in text.split():
        if word.lower() not in stop_words:
            new_text.append(word)
    return " ".join(new_text)


In [187]:
remove_stopwords("the quick brown fox jumps over the lazy dog. I am Avishek Basyal. My is your")

'quick brown fox jumps lazy dog. Avishek Basyal.'

In [188]:
df['review'].apply(remove_stopwords)

0        one reviewers mentioned watching 1 oz episode ...
1        wonderful little production. filming technique...
2        thought wonderful way spend time hot summer we...
3        basically there's family little boy (jake) thi...
4        petter mattei's "love time money" visually stu...
                               ...                        
49995    thought movie right good job. creative origina...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    one expects star trek movies high art, fans ex...
Name: review, Length: 50000, dtype: object

## 8) Handling emojis


In [189]:

import re

def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"  # Miscellaneous symbols and pictographs
        "]+",
        flags=re.UNICODE
    )

    return emoji_pattern.sub(r'', text)



In [190]:
remove_emoji("what was that Lmao 😂😂😂")

'what was that Lmao '

In [191]:
import emoji
print(emoji.demojize( "python is 💕" ) )
print(emoji.demojize( "PUshpa is 🔥" ) )

python is :two_hearts:
PUshpa is :fire:


# 9) Tokenization

## Different methods of tokenization


## 9.1) Using the python split function

In [192]:
# word tokenization

sent1 = "I am going to college"
sent1.split()

['I', 'am', 'going', 'to', 'college']

In [193]:
# sentence tokenization

sent2 = "Heollo everyone. I am living in my room. My name is Avishek Basyal"
sent2.split('.')

['Heollo everyone', ' I am living in my room', ' My name is Avishek Basyal']

In [194]:
## problem with split function

sent3 = "I am loving Nepal!" # it also tokenize the exclamation with 'Nepal!' . in future if the word 'Nepal' without exclamation appears , it understand both as differenc

sent3.split(' ')

['I', 'am', 'loving', 'Nepal!']

In [195]:
sent4 = 'Where do think I should go? I have 3 day holiday'
sent4.split('.')  # split function can split only one thing . here it can't split question mark as questin mark seprates two sentences

['Where do think I should go? I have 3 day holiday']

## 9.2) Using Regular function


In [196]:
import re
tokens = re.findall("[\w']+",sent3)
tokens  # here it just removes the punctuation marks from sentence 3

['I', 'am', 'loving', 'Nepal']

## 9.3) NLTK

In [197]:
from nltk.tokenize import word_tokenize , sent_tokenize

In [198]:
word_tokenize(sent3) # it also tokenize the exclamation seprately

['I', 'am', 'loving', 'Nepal', '!']

In [199]:
text = """Lorem Ipsum is simply dummy text of the printing and typesetting industry?
Lorem Ipsum has been the industry's standard dummy text ever since the 1500s,
when an unknown printer took a galley of type and scrambled it to make a type specimen book."""

sent_tokenize(text)

['Lorem Ipsum is simply dummy text of the printing and typesetting industry?',
 "Lorem Ipsum has been the industry's standard dummy text ever since the 1500s,\nwhen an unknown printer took a galley of type and scrambled it to make a type specimen book."]

In [200]:
sent5 = 'I have a Ph.D in A.I'
sent6 ="We're here to help! mail us at nks@gmail.com"
sent7 = "A 5km ride cost $0.50"

In [201]:
word_tokenize(sent5)

['I', 'have', 'a', 'Ph.D', 'in', 'A.I']

In [202]:
word_tokenize(sent6) # it seprates we're as : 'we' , "'re'" and it also seprates the gmail also

['We',
 "'re",
 'here',
 'to',
 'help',
 '!',
 'mail',
 'us',
 'at',
 'nks',
 '@',
 'gmail.com']

In [203]:
word_tokenize(sent7)

['A', '5km', 'ride', 'cost', '$', '0.50']

## 9.4) spacy -> Better than other generally , but every methods make mistakes

In [204]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [205]:
doc1 = nlp(sent5)
doc2 = nlp(sent6)
doc3 = nlp(sent7)


In [206]:
for token in doc1:
    print (token)  # here it seprates Ph.D into two seprate tokens

I
have
a
Ph
.
D
in
A.I


In [207]:
for token in doc2:
    print (token) # it writes email perfectly but make mistake in "we're"

We
're
here
to
help
!
mail
us
at
nks@gmail.com


In [208]:
for token in doc3:
    print (token)  # works perfectly for this

A
5
km
ride
cost
$
0.50


## 10) Stemming


In [209]:

from nltk.stem import PorterStemmer

ps = PorterStemmer()

def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()]) # # Splits the text into words, applies stemming to each word, and joins them back with a space.

In [210]:
sample = "walk walks walking walked"
stem_words(sample)

'walk walk walk walk'

In [211]:
text = 'probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble cause but its not preachy or boring it just never gets olc'
stem_words(text)

'probabl my alltim favorit movi a stori of selfless sacrific and dedic to a nobl caus but it not preachi or bore it just never get olc'

## 11) Lemmatization

In [212]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

sentence = "He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun."

punctuations = '?:!.,'

sentence_words = nltk.word_tokenize(sentence)

for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)

sentence_words 

print('{:<20} {:<20}'.format("Word", "Lemma"))
for word in sentence_words:
    print('{:<20} {:<20}'.format(word, wordnet_lemmatizer.lemmatize(word , pos='v'))) # you have to specify part of speech

Word                 Lemma               
He                   He                  
was                  be                  
running              run                 
and                  and                 
eating               eat                 
at                   at                  
same                 same                
time                 time                
He                   He                  
has                  have                
bad                  bad                 
habit                habit               
of                   of                  
swimming             swim                
after                after               
playing              play                
long                 long                
hours                hours               
in                   in                  
the                  the                 
Sun                  Sun                 
